In [2]:
import pandas as pd
import geopandas as gpd


ROOT_DATA = "./data/raw"

In [3]:
df_acc = pd.read_csv(f"{ROOT_DATA}/accidente-de-trafico-en-bogota-entre-2007-y-2017-geopoint.csv", sep=';')
df_vm = pd.read_csv(f"{ROOT_DATA}/vm_acc_via.csv", sep=';')

print(f"Datos totales en Accidente de Tráfico: {len(df_acc)}")
print(f"Datos totales en vm_acc_via: {len(df_vm)}")

Datos totales en Accidente de Tráfico: 330063
Datos totales en vm_acc_via: 396244


## Count duplicated values

In [17]:
# Duplicados en código de accidente
duplicados_acc = df_acc['CODIGO_ACCIDENTE'].duplicated().sum()
print(f"Duplicados de CODIGO_ACCIDENTE en df_acc: {duplicados_acc}")

duplicados_vm = df_vm['CODIGO_ACCIDENTE'].duplicated().sum()
print(f"Duplicados de CODIGO_ACCIDENTE en df_vm: {duplicados_vm}")

Duplicados de CODIGO_ACCIDENTE en df_acc: 0
Duplicados de CODIGO_ACCIDENTE en df_vm: 75005


In [18]:
# Duplicados en formulario
duplicados_acc_for = df_acc['FORMULARIO'].duplicated().sum()
print(f"Duplicados de FORMULARIO en df_acc: {duplicados_acc_for}")

duplicados_vm_for = df_vm['FORMULARIO'].duplicated().sum()
print(f"Duplicados de FORMULARIO en df_vm: {duplicados_vm_for}")

Duplicados de FORMULARIO en df_acc: 0
Duplicados de FORMULARIO en df_vm: 75005


In [19]:
# Duplicados de OBJECTID
duplicados_vm_obj = df_vm['OBJECTID'].duplicated().sum()
print(f"Duplicados de OBJECTID en df_vm: {duplicados_vm_obj}")

Duplicados de OBJECTID en df_vm: 0


In [20]:
# Duplicados en df_vm
duplicates_vm = df_vm.duplicated(subset=['FORMULARIO','CODIGO_ACCIDENTE'], keep=False)
print(f"Cantidad de duplicados de FORMULARIO y CODIGO_ACCIDENTE en df_vm: {duplicados_vm.sum()}")

Cantidad de duplicados de FORMULARIO y CODIGO_ACCIDENTE en df_vm: 75005


In [21]:
# Joins
df_merged_objectID = pd.merge(df_acc, df_vm, on='OBJECTID', how='inner')
df_merged_formulario = pd.merge(df_acc, df_vm, on='FORMULARIO', how='inner')
df_merged_cod_accidente = pd.merge(df_acc, df_vm, on='CODIGO_ACCIDENTE', how='inner')

print(f"Número de registros en el DataFrame resultante unido por objectID: {len(df_merged_objectID)}")
print(f"Número de registros en el DataFrame resultante unido por formulario: {len(df_merged_formulario)}")
print(f"Número de registros en el DataFrame resultante unido por codigo_accidente: {len(df_merged_cod_accidente)}")


Número de registros en el DataFrame resultante unido por objectID: 179310
Número de registros en el DataFrame resultante unido por formulario: 396244
Número de registros en el DataFrame resultante unido por codigo_accidente: 396244


In [22]:
print(f"Accidentes con datos de vía: {df_merged_cod_accidente['CODIGO_VIA'].notna().sum()}")
print(f"Accidentes sin datos de vía: {df_merged_cod_accidente['CODIGO_VIA'].isna().sum()}")
df_merged_cod_accidente.columns

Accidentes con datos de vía: 396244
Accidentes sin datos de vía: 0


Index(['Geo Point', 'OBJECTID_x', 'FORMULARIO_x', 'CODIGO_ACCIDENTE',
       'FECHA_OCURRENCIA', 'HORA_OCURRENCIA', 'ANO_OCURRENCIA',
       'MES_OCURRENCIA', 'DIA_OCURRENCIA', 'DIRECCION', 'GRAVEDAD', 'CLASE',
       'LOCALIDAD', 'AREAS_TRANSITO', 'CUADRANTE_TRANSITO', 'MUNICIPIO',
       'TEXTO_OBSERVACIONES', 'FECHA_HORA', 'OBJECTID_y', 'FORMULARIO_y',
       'CODIGO_VIA', 'GEOMETRICA_A', 'GEOMETRICA_B', 'GEOMETRICA_C',
       'UTILIZACION', 'CALZADAS', 'CARRILES', 'MATERIAL', 'ESTADO',
       'CONDICIONES', 'ILUMINACION_A', 'ILUMINACION_B', 'AGENTE_TRANSITO',
       'SEMAFORO', 'VISUAL', 'CODIGO'],
      dtype='object')

In [23]:
# Filas donde FORMULARIO_x y FORMULARIO_y son diferentes (y FORMULARIO_y no es nulo)
diff_formularios = df_merged_cod_accidente[df_merged_cod_accidente['FORMULARIO_x'] != df_merged_cod_accidente['FORMULARIO_y']]

print(f"Cantidad de registros donde FORMULARIO difiere: {len(diff_formularios)}")

Cantidad de registros donde FORMULARIO difiere: 0


In [24]:
# Eliminar una columna de formularios
df_merged_cod_accidente.drop('FORMULARIO_y', axis=1, inplace=True)
df_merged_cod_accidente.rename(columns={'FORMULARIO_x': 'FORMULARIO'}, inplace=True)

df_merged_cod_accidente.columns

Index(['Geo Point', 'OBJECTID_x', 'FORMULARIO', 'CODIGO_ACCIDENTE',
       'FECHA_OCURRENCIA', 'HORA_OCURRENCIA', 'ANO_OCURRENCIA',
       'MES_OCURRENCIA', 'DIA_OCURRENCIA', 'DIRECCION', 'GRAVEDAD', 'CLASE',
       'LOCALIDAD', 'AREAS_TRANSITO', 'CUADRANTE_TRANSITO', 'MUNICIPIO',
       'TEXTO_OBSERVACIONES', 'FECHA_HORA', 'OBJECTID_y', 'CODIGO_VIA',
       'GEOMETRICA_A', 'GEOMETRICA_B', 'GEOMETRICA_C', 'UTILIZACION',
       'CALZADAS', 'CARRILES', 'MATERIAL', 'ESTADO', 'CONDICIONES',
       'ILUMINACION_A', 'ILUMINACION_B', 'AGENTE_TRANSITO', 'SEMAFORO',
       'VISUAL', 'CODIGO'],
      dtype='object')

## Save dataset

This sections involves, making an inner join with `df_acc` and `df_vm` on `OBJECTID`. And dealing with nan values

Would be great try to complete the NaN data using the historical data. For example, column "UTILIZATION" related to the principal feature of the road where the accident happened, maybe there are data at the same road, at the same month or week happened another accident but this one with data. So, theorically, it's possible to complete missing data, but it's necessary to know all the details, for example, if there was a construction, and all that kind of things. It is just an idea, it's good to know all the possibilities

In [5]:
df = (
    pd.merge(df_acc, df_vm, on="OBJECTID", how="left")
    .drop(columns=["FORMULARIO_y", "CODIGO_ACCIDENTE_y"])
    .rename(
        columns={"FORMULARIO_x": "FORMULARIO", "CODIGO_ACCIDENTE_x": "CODIGO_ACCIDENTE"}
    )
)

# Parse geo points
geopoints = df["Geo Point"].str.split(",", expand=True).astype(float)
geopoints.columns = ["latitude", "longitude"]

# Combine dataframes
df = pd.concat([df, geopoints], axis=1)

In [6]:
df.head()

,Geo Point,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,FECHA_OCURRENCIA,HORA_OCURRENCIA,ANO_OCURRENCIA,MES_OCURRENCIA,DIA_OCURRENCIA,DIRECCION,...,ESTADO,CONDICIONES,ILUMINACION_A,ILUMINACION_B,AGENTE_TRANSITO,SEMAFORO,VISUAL,CODIGO,latitude,longitude
0,"4.655401615, -74.101699558",646545,A00410124,306211,Fri Sep 05 00:00:00 CEST 2008,12:50:00,2008,SEPTIEMBRE,VIERNES,AK 68-44 02,...,BUENO,SECA,SIN,BUENA,NaN,NaN,NaN,646545-298258-1,4.655402,-74.101700
1,"4.68941675, -74.109350811",563782,839933000,291565,Tue May 20 00:00:00 CEST 2008,23:40:00,2008,MAYO,MARTES,AV CIUDAD DE CALI-CL 66 02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.689417,-74.109351
2,"4.689749817, -74.10901617",4398952,A000038718,4398952,Thu Dec 04 00:00:00 CET 2014,17:30:00,2014,DICIEMBRE,JUEVES,AV AVENIDA CIUDAD DE CALI-CL 66A 02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.689750,-74.109016
3,"4.615128763, -74.108947186",1612,726750500,30243,Fri Jun 29 00:00:00 CEST 2007,19:30:00,2007,JUNIO,VIERNES,TR 40A-5 32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.615129,-74.108947
4,"4.646695747, -74.108791627",569692,840564600,297475,Mon Jul 21 00:00:00 CEST 2008,07:30:00,2008,JULIO,LUNES,KR 68-22A 02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.646696,-74.108792


In [7]:
df["SEMAFORO"].unique()

array([nan, 'OPERANDO', 'NINGUNO', 'APAGADO', 'CON DAÑOS', 'OCULTO',
       'INTERMITENTE'], dtype=object)

In [8]:
df.isna().sum()

Geo Point                   0
OBJECTID                    0
FORMULARIO                  0
CODIGO_ACCIDENTE            0
FECHA_OCURRENCIA            0
HORA_OCURRENCIA             0
ANO_OCURRENCIA              0
MES_OCURRENCIA              0
DIA_OCURRENCIA              0
DIRECCION                   0
GRAVEDAD                    3
CLASE                     183
LOCALIDAD                  11
AREAS_TRANSITO             14
CUADRANTE_TRANSITO       1754
MUNICIPIO                   0
TEXTO_OBSERVACIONES    316231
FECHA_HORA                  0
CODIGO_VIA             150753
GEOMETRICA_A           151459
GEOMETRICA_B           153204
GEOMETRICA_C           154436
UTILIZACION            152301
CALZADAS               152949
CARRILES               152028
MATERIAL               152188
ESTADO                 152576
CONDICIONES            151823
ILUMINACION_A          164995
ILUMINACION_B          254915
AGENTE_TRANSITO        176173
SEMAFORO               277944
VISUAL                 329583
CODIGO    

### Format datetime

I was trying to use this format `%a %b %d %H:%M:%S %Z %Y` in "FECHA_OCURRENCIA", but there are still errors, so I'll format by my own the datetime as `YYYY-MM-DD HH:mm:ss`

In [9]:
month_dict = {
    "ENERO": 1,
    "FEBRERO": 2,
    "MARZO": 3,
    "ABRIL": 4,
    "MAYO": 5,
    "JUNIO": 6,
    "JULIO": 7,
    "AGOSTO": 8,
    "SEPTIEMBRE": 9,
    "OCTUBRE": 10,
    "NOVIEMBRE": 11,
    "DICIEMBRE": 12,
}

day_week_dict = {
    "LUNES": 1,
    "MARTES": 2,
    "MIERCOLES": 3,
    "JUEVES": 4,
    "VIERNES": 5,
    "SABADO": 6,
    "DOMINGO": 7,
}

In [10]:
# Convert as a number the month and day of the week
df.rename(columns={"DIA_OCURRENCIA": "DIA_SEMANA_OCURRENCIA"}, inplace=True)
df["DIA_SEMANA_OCURRENCIA"] = df["DIA_SEMANA_OCURRENCIA"].apply(
    lambda x: day_week_dict[x] if isinstance(x, str) else x
)
df["MES_OCURRENCIA"] = df["MES_OCURRENCIA"].apply(
    lambda x: month_dict[x] if isinstance(x, str) else x
)

# Days of the event
days = list(map(lambda x: int(x[2]), df["FECHA_OCURRENCIA"].str.split(" ").tolist()))
df["DIA_OCURRENCIA"] = days

# Format datetime
datetimes = []
for _, row in df.iterrows():
    day = row["DIA_OCURRENCIA"]
    month = row["MES_OCURRENCIA"]
    year = row["ANO_OCURRENCIA"]
    hour = row["HORA_OCURRENCIA"]
    dt = f"{year}-{month:02d}-{day:02d} {hour}"
    datetimes.append(dt)

df["FECHA_OCURRENCIA"] = pd.to_datetime(datetimes)

In [11]:
display(
    df[
        [
            "FECHA_OCURRENCIA",
            "HORA_OCURRENCIA",
            "ANO_OCURRENCIA",
            "MES_OCURRENCIA",
            "DIA_OCURRENCIA",
            "DIA_SEMANA_OCURRENCIA",
        ]
    ].head(),
    df["MES_OCURRENCIA"].unique(),
    df["DIA_OCURRENCIA"].unique(),
    # pd.to_datetime(df["FECHA_OCURRENCIA"], format="%a %b %d %H:%M:%S %Z %Y")
)

,FECHA_OCURRENCIA,HORA_OCURRENCIA,ANO_OCURRENCIA,MES_OCURRENCIA,DIA_OCURRENCIA,DIA_SEMANA_OCURRENCIA
0,2008-09-05 12:50:00,12:50:00,2008,9,5,5
1,2008-05-20 23:40:00,23:40:00,2008,5,20,2
2,2014-12-04 17:30:00,17:30:00,2014,12,4,4
3,2007-06-29 19:30:00,19:30:00,2007,6,29,5
4,2008-07-21 07:30:00,07:30:00,2008,7,21,1


array([ 9,  5, 12,  6,  7,  4, 10,  3,  1,  2,  8, 11])

array([ 5, 20,  4, 29, 21,  6,  1, 22, 10, 31,  8, 13, 23,  7, 12, 11, 14,
       16, 24, 25, 15, 19, 30, 28, 17,  3, 18,  2,  9, 27, 26])

### Locality related

Some points in df_acc have nan values in the locality. This is because those points are not in Bogotá, but in Soacha and El Porvenir. Anyway, there are only 11 values, so we can skip them

In [12]:
# Replace bad locality names
df = df.replace({"ANTONIO NARINO": "ANTONIO NARIÑO", "CANDELARIO": "CANDELARIA"})

In [13]:
# Concatenate localities geometry
localities = gpd.read_file(f"{ROOT_DATA}/poligonos-localidades.zip").to_crs("EPSG:4326")

gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326",  # WGS84
)

print(localities.columns)

Index(['Nombre_de_l', 'Acto_admini', 'Area_de_la_', 'Identificad', 'geometry'], dtype='object')


In [14]:
nan_localities = gdf_points[gdf_points["LOCALIDAD"].isna()]

# Just in case there are points inside Bogota
# with nan locality (it's not the case)
for idx, row in nan_localities.copy().iterrows():
    point = row["geometry"]
    for _, loc in localities.iterrows():
        if not loc["geometry"].contains(point):
            continue
        print(idx, loc["Nombre_de_l"])

        # Assign the found locality name
        gdf_points.at[idx, "LOCALIDAD"] = loc["Nombre_de_l"]

        # Remove row from nan_localities
        nan_localities.drop(idx, inplace=True)
        break


nan_localities_indexes = nan_localities.index.tolist()
print(f"Total nan localities to drop: {len(nan_localities_indexes)}")
print(nan_localities_indexes)
df.drop(nan_localities_indexes, inplace=True)

Total nan localities to drop: 11
[16362, 29130, 76158, 85823, 121104, 164473, 179264, 184724, 194069, 299525, 323560]


### `GRAVEDAD`& `CLASE`

These columns are important to visualize, and they are only 3 and 183 for `GRAVEDAD` and `CLASE` respectively. So, we can skip them and shouldn't be a big deal

In [15]:
display(df.shape, df.dropna(subset=["GRAVEDAD", "CLASE"]).shape)

(330052, 37)

(329866, 37)

In [16]:
df.dropna(subset=["GRAVEDAD", "CLASE"], inplace=True)

### Save finally

In [17]:
# Drop unnecessary columns
df.drop(
    columns=[
        # Duplicated with lat/lon
        "Geo Point",
        "DIRECCION",  # Duplicated in some way

        # Duplicated with "FECHA_OCURRENCIA"
        "HORA_OCURRENCIA",
        "ANO_OCURRENCIA",
        "MES_OCURRENCIA",
        "DIA_OCURRENCIA",

        # Not relevant
        "TEXTO_OBSERVACIONES",
        "CODIGO_VIA",
        "CODIGO",
        "MUNICIPIO",  # It's always BOGOTA
        # Not relevant for this analysis. These columns
        # are related to the transit authority
        # that attended the accident
        "AREAS_TRANSITO",
        "CUADRANTE_TRANSITO",
        "AGENTE_TRANSITO",  # I'm not sure what's the meaning of this one tbh
    ],
    inplace=True,
    errors="ignore",
)

display(df.columns, df.shape)

Index(['OBJECTID', 'FORMULARIO', 'CODIGO_ACCIDENTE', 'FECHA_OCURRENCIA',
       'DIA_SEMANA_OCURRENCIA', 'GRAVEDAD', 'CLASE', 'LOCALIDAD', 'FECHA_HORA',
       'GEOMETRICA_A', 'GEOMETRICA_B', 'GEOMETRICA_C', 'UTILIZACION',
       'CALZADAS', 'CARRILES', 'MATERIAL', 'ESTADO', 'CONDICIONES',
       'ILUMINACION_A', 'ILUMINACION_B', 'SEMAFORO', 'VISUAL', 'latitude',
       'longitude'],
      dtype='object')

(329866, 24)

In [18]:
df.to_csv("./data/processed/dataset.csv", index=False)

## Read processed dataset

In [19]:
pd.read_csv("./data/processed/dataset.csv")

,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,FECHA_OCURRENCIA,DIA_SEMANA_OCURRENCIA,GRAVEDAD,CLASE,LOCALIDAD,FECHA_HORA,GEOMETRICA_A,...,CARRILES,MATERIAL,ESTADO,CONDICIONES,ILUMINACION_A,ILUMINACION_B,SEMAFORO,VISUAL,latitude,longitude
0,646545,A00410124,306211,2008-09-05 12:50:00,5,CON HERIDOS,ATROPELLO,TEUSAQUILLO,Fri Sep 05 12:49:59 CEST 2008,RECTA,...,DOS,CONCRETO,BUENO,SECA,SIN,BUENA,NaN,NaN,4.655402,-74.101700
1,563782,839933000,291565,2008-05-20 23:40:00,2,SOLO DANOS,CHOQUE,ENGATIVA,Tue May 20 23:40:00 CEST 2008,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.689417,-74.109351
2,4398952,A000038718,4398952,2014-12-04 17:30:00,4,SOLO DANOS,CHOQUE,ENGATIVA,Thu Dec 04 17:30:00 CET 2014,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.689750,-74.109016
3,1612,726750500,30243,2007-06-29 19:30:00,5,CON HERIDOS,ATROPELLO,PUENTE ARANDA,Fri Jun 29 19:30:00 CEST 2007,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.615129,-74.108947
4,569692,840564600,297475,2008-07-21 07:30:00,1,SOLO DANOS,CHOQUE,TEUSAQUILLO,Mon Jul 21 07:30:00 CEST 2008,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.646696,-74.108792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329861,1707169,A00719849,352633,2010-03-04 14:30:00,4,SOLO DANOS,CHOQUE,FONTIBON,Thu Mar 04 14:30:00 CET 2010,RECTA,...,CUATRO O MAS,ASFALTO,BUENO,SECA,SIN,NaN,NaN,NaN,4.671688,-74.145705
329862,9682,712264000,16686,2007-03-07 11:00:00,3,SOLO DANOS,CHOQUE,CIUDAD BOLIVAR,Wed Mar 07 11:00:00 CET 2007,RECTA,...,TRES O MAS,ASFALTO,BUENO,SECA,SIN,NaN,OPERANDO,NaN,4.549462,-74.154415
329863,4421360,A12506,4421360,2015-08-19 16:50:00,3,SOLO DANOS,CHOQUE,CIUDAD BOLIVAR,Wed Aug 19 16:50:00 CEST 2015,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.596006,-74.158040
329864,60219,828721300,673,2008-01-05 19:27:00,6,SOLO DANOS,CHOQUE,TEUSAQUILLO,Sat Jan 05 19:27:00 CET 2008,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.645298,-74.086861
